In [ ]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import re
import csv
import nltk
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np, pylab as pl
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pymysql
import pymysql.cursors


In [ ]:
# dummy values 
conn = pymysql.connect(host='localhost', port=3306, user='root', use_unicode=True, passwd='admin1234', db='tweet',charset='utf8')
cur = conn.cursor()

try:
    with conn.cursor() as cursor:
        query = "SELECT * FROM tweetdata "
        df = pd.read_sql(query, conn)
finally:
    conn.close()

In [ ]:
df.describe()

In [ ]:
#traffic,jammed,

#df["label"] = "" #lavel column assign
df['text'] = df['text'].str.replace('@',' ') #remove @ 
df = df[df.text.str.contains("rt" or "RT") == False] #remove RT

remlink = []
for i in df['text']:
    a = re.sub(r"http\S+", "", i)
    remlink.append(a)
    
df['text'] = remlink #update TEXT after remove LINK


In [ ]:
# New df for remove tweet not according to keywords

dfkeyword = df[df.text.str.contains("hujan | traffic slow| slow | traffic jammed | accident | stuck")]
dfkeyword=dfkeyword.reset_index()
#dfkeyword = df[df.text.str.contains("makan") == True]
#dfkeyword = df[df.text.str.contains("hujan") == True]

del dfkeyword['index'] #del default index col

In [ ]:
dfkeyword

In [54]:
# Malay Stop Word ONLY


stop_words = set({'ada','inikah','sampai','adakah','inilah','sana','adakan','itu','sangat','adalah','itukah','sangatlah','adanya','itulah','saya','adapun','jadi','se','agak','jangan','seandainya','agar','janganlah','sebab','akan','jika','sebagai','aku','jikalau','sebagaimana','akulah','jua','sebanyak','akupun','juapun','sebelum','al','juga','sebelummu','alangkah','kalau','sebelumnya','allah','kami','sebenarnya','amat','kamikah','secara','antara','kamipun','sedang','antaramu','kamu','sedangkan','antaranya','kamukah','sedikit','apa','kamupun','sedikitpun','apa-apa','katakan','segala','apabila','ke','sehingga','apakah','kecuali','sejak','apapun','kelak','sekalian','atas','kembali','sekalipun','atasmu','kemudian','sekarang','atasnya','kepada','sekitar','atau','kepadaku','selain','ataukah','kepadakulah','selalu','ataupun','kepadamu','selama','bagaimana','kepadanya','selama-lamanya','bagaimanakah','kepadanyalah','seluruh','bagi','kerana','seluruhnya','bagimu','kerananya','sementara','baginya','kesan','semua','bahawa','ketika','semuanya','bahawasanya','kini','semula','bahkan','kita','senantiasa','bahwa','ku','sendiri','banyak','kurang','sentiasa','banyaknya','lagi','seolah','barangsiapa','lain','seolah-olah','bawah','lalu','seorangpun','beberapa','lamanya','separuh','begitu','langsung','sepatutnya','begitupun','lebih','seperti','belaka','maha','seraya','belum','mahu','sering','belumkah','mahukah','serta','berada','mahupun','seseorang','berapa','maka','sesiapa','berikan','malah','sesuatu','beriman','mana','sesudah','berkenaan','manakah','sesudahnya','berupa','manapun','sesungguhnya','beserta','masih','sesungguhnyakah','biarpun','masing','setelah','bila','masing-masing','setiap','bilakah','melainkan','siapa','bilamana','memang','siapakah','bisa','mempunyai','sini','boleh','mendapat','situ','bukan','mendapati','situlah','bukankah','mendapatkan','suatu','bukanlah','mengadakan','sudah','dahulu','mengapa','sudahkah','dalam','mengapakah','sungguh','dalamnya','mengenai','sungguhpun','dan','menjadi','supaya','dapat','menyebabkan','tadinya','dapati','menyebabkannya','tahukah','dapatkah','mereka','tak','dapatlah','merekalah','tanpa','dari','merekapun','tanya','daripada','meskipun','tanyakanlah','daripadaku','mu','tapi','daripadamu','nescaya','telah','daripadanya','niscaya','tentang','demi','nya','tentu','demikian','olah','terdapat','demikianlah','oleh','terhadap','dengan','orang','terhadapmu','dengannya','pada','termasuk','di','padahal','terpaksa','dia','padamu','tertentu','dialah','padanya','tetapi','didapat','paling','tiada','didapati','para','tiadakah','dimanakah','pasti','tiadalah','engkau','patut','tiap','engkaukah','patutkah','tiap-tiap','engkaulah','per','tidak','engkaupun','pergilah','tidakkah','hai','perkara','tidaklah','hampir','perkaranya','turut','hampir-hampir','perlu','untuk','hanya','pernah','untukmu','hanyalah','pertama','wahai','hendak','pula','walau','hendaklah','pun','walaupun','hingga','sahaja','ya','ia','saja','yaini','iaitu','saling','yaitu','ialah','sama','yakni','ianya','yang','inginkah','samakah','ini','sambil'})


c = []

for word in dfkeyword['text']:
    word=word.lower()
    word_tokens = word_tokenize(word)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    if word not in stop_words:        
        filtered_sentence.append(word)
        a=" ".join( filtered_sentence )
        c.append(a)

    
dfkeyword['text'] = c

In [55]:
#Insert class label (1 - yes | 0 - no)

yn =[]
count=1
for i in dfkeyword['text'] :
    print("{} : {}".format(count,i))
    yon = int(input("1 ... 0"))
    print("\n")
    yn.append(yon)
    count+=1
    
dfkeyword['label'] = yn

1 : pahal la lemah sgt antibody ni . kene hujan sikit dh selsema la , demam la pahal la lemah sgt antibody ni . kene hujan sikit dh selsema la , demam la pahal la lemah sgt antibody ni . kene hujan sikit dh selsema la , demam la pahal la lemah sgt antibody ni. kene hujan sikit dh selsema la, demam la
1 ... 00


2 : semalam cari baju hujan dekat petron , kat cashier baju hujan ? tunjuk arah peti minuman . blur… semalam cari baju hujan dekat petron , kat cashier baju hujan ? tunjuk arah peti minuman . blur… semalam cari baju hujan dekat petron , kat cashier baju hujan ? tunjuk arah peti minuman . blur… semalam cari baju hujan dekat petron, tanya kat cashier ada baju hujan tak? dia tunjuk arah peti minuman. aku blur… 
1 ... 01




In [56]:
dfkeyword.to_csv('data/tweet-label-class.csv', encoding='utf-8',header=True)

In [ ]:
cur.execute("UPDATE tweetdata SET  (`text`)=%s, (`label`)=%s ", (dfkeyword['text'], dfkeyword['label']))

cur.commit()
conn.close()